In [1]:
import numpy
import optuna
from pycbc.filter import matched_filter
from pycbc.waveform import get_td_waveform
from pycbc.catalog import Merger
from pycbc.filter import resample_to_delta_t, highpass
from pycbc.psd import interpolate, inverse_spectrum_truncation

/home/susanna.green/.conda/envs/PyCBCandOptuna/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
merger = Merger("GW170814") #GW150914

# Get the data from the Hanford detector
strain = merger.strain('L1')

# Remove the low frequency content and downsample the data to 2048Hz
strain = resample_to_delta_t(highpass(strain, 15.0), 1.0/2048)

# Remove 2 seconds of data from both the beginning and end
conditioned = strain.crop(2, 2)

# Estimate the power spectral density
# We use 4 second samles of our time series in Welch method.
psd = conditioned.psd(4)

# Now that we have the psd we need to interpolate it to match our data
# and then limit the filter length of 1 / PSD. After this, we can
# directly use this PSD to filter the data in a controlled manner

psd = interpolate(psd, conditioned.delta_f)

# 1/PSD will now act as a filter with an effective length of 4 seconds
# Since the data has been highpassed above 15 Hz, and will have low values
# below this we need to informat the function to not include frequencies
# below this frequency. 
psd = inverse_spectrum_truncation(psd, int(4 * conditioned.sample_rate),low_frequency_cutoff=15)

In [6]:
#Optuna gnerating the template. 
def objective(trial):
    m1 = trial.suggest_float('m1', 5, 100)
    m2 = trial.suggest_float('m2', 5, 100)
    hp, hc = get_td_waveform(approximant="SEOBNRv4_opt",mass1=m1, mass2=m2, delta_t=conditioned.delta_t,f_lower=20)
    hp.resize(len(conditioned))
    template = hp.cyclic_time_shift(hp.start_time)
    snr = matched_filter(template, conditioned, psd=psd, low_frequency_cutoff=20)
    snr = snr.crop(4 + 4, 4)
    peak = abs(snr).numpy().argmax()
    snrp = abs(snr[peak])
    return float(snrp)


In [7]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=500)
print(study.best_params)

[I 2023-03-01 12:48:09,969] A new study created in memory with name: no-name-a3346d87-3872-4dc3-8945-b0c7b439255e
[I 2023-03-01 12:48:10,113] Trial 0 finished with value: 7.698066011087465 and parameters: {'m1': 55.56547943266378, 'm2': 70.72894784951535}. Best is trial 0 with value: 7.698066011087465.
[I 2023-03-01 12:48:10,272] Trial 1 finished with value: 7.634648906819682 and parameters: {'m1': 79.03471719647892, 'm2': 72.62657332069207}. Best is trial 0 with value: 7.698066011087465.
[I 2023-03-01 12:48:10,390] Trial 2 finished with value: 7.909896299752217 and parameters: {'m1': 39.6000251167725, 'm2': 50.1651401640235}. Best is trial 2 with value: 7.909896299752217.
[I 2023-03-01 12:48:10,538] Trial 3 finished with value: 8.145804482435437 and parameters: {'m1': 53.16931336852443, 'm2': 31.308655990993273}. Best is trial 3 with value: 8.145804482435437.
[I 2023-03-01 12:48:10,722] Trial 4 finished with value: 6.276415912259633 and parameters: {'m1': 12.31024672644979, 'm2': 11.2

{'m1': 30.424183490466486, 'm2': 30.269993497742043}
